In [8]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.optimize import minimize




In [3]:
#write program of Runge-Kutta order 4.5 (Dorman Prince) method to solve the differential equation
def f(x,y):
    return (x-y)/2
def RK45(x0,y0,xn,n):
    h=(xn-x0)/n
    print('\n--------SOLUTION--------')
    print('x0\ty0\tk1\tk2\tk3\tk4\tk5\tk6\ty1')
    print('--------------------------------------------------------------------------------')
    for i in range(n):
        k1=h*f(x0,y0)
        k2=h*f(x0+h/5,y0+k1/5)
        k3=h*f(x0+3*h/10,y0+3*k1/40+9*k2/40)
        k4=h*f(x0+4*h/5,y0+44*k1/45-56*k2/15+32*k3/9)
        k5=h*f(x0+8*h/9,y0+19372*k1/6561-25360*k2/2187+64448*k3/6561-212*k4/729)
        k6=h*f(x0+h,y0+9017*k1/3168-355*k2/33+46732*k3/5247+49*k4/176-5103*k5/18656)
        y=y0+35*k1/384+500*k3/1113+125*k4/192-218

In [4]:
#Lodka-Volterra non linear model
def MakeModel(Alpha,Beta,Gamma,Delta,Variables):
    Equation1=Variables[0]*(Alpha-Beta*Variables[1])
    Equation2=-Variables[1]*(Gamma-Delta*Variables[0])
    return [Equation1,Equation2]

#Integration time
SolverTime=np.linspace(0,50,num=200)

#Parameters of the model 
alpha=0.25
beta=0.55
gamma=0.3
delta=0.6

#Initial conditions 
Int=np.array([3,1])  

#Model Solution 
def ODEModel(InitialConditions,t):
    return MakeModel(alpha,beta,gamma,delta,InitialConditions)

Solution=odeint(ODEModel,Int,SolverTime)

In [5]:
#Measures the squared error between the data and the fit given an initial parameter guess
def SquaredError(InitialParameterGuess):
    
    try:
        #Some guesses will result in an error or an invalid integration 
        #this ensures that the error is estimated only for valid solutions 
        cModelParams=curve_fit(ModelFit,SolverTime,WhiteSignal,p0=InitialParameterGuess)
        cSolution=ModelSolver(SolverTime,cModelParams[0][0],cModelParams[0][1],cModelParams[0][2],cModelParams[0][3],Int)
    
        error=[(val-sal)**2 for val,sal in zip(cSolution,WhiteSignal)]
        
    except RuntimeError:
        
        #Based on the scale of the data the following list of values will be large enough to be rejected as a solution 
        error=[10,10]
    
    return sum(error)

In [6]:
#Random selection of the initial guess values 
def RandomSearch(Iterations):

    nIt=Iterations 
    GuessContainer=[]
    ErrorContainer=[]
    for k in range(nIt):
        lGuess=[np.random.uniform(low=0,high=1) for val in range(4)]
        lerror=SquaredError(lGuess)
        GuessContainer.append(lGuess) #Contains the list of random initial values 
        ErrorContainer.append(lerror) #List of the errors 
    minError=np.min(ErrorContainer) #Min error value 
    minLocation=[j for j in range(nIt) if ErrorContainer[j]==minError] #Returns the location of the min value in the list
    bestGuess=GuessContainer[minLocation[0]] #Best initial value guess 
    
    return ErrorContainer,bestGuess

In [9]:
#Vector of random initial values 
g0=[np.random.uniform(low=0,high=1) for val in range(4)] 

#Minimization of the squared error by Nelder-Mead 
res = minimize(SquaredError, g0, method='nelder-mead',options={'xtol': 1e-3, 'maxiter':100,'disp': False})

ModelParams03=curve_fit(ModelFit,SolverTime,WhiteSignal,p0=res.x)
FitSolution3=ModelSolver(SolverTime,ModelParams03[0][0],ModelParams03[0][1],ModelParams03[0][2],ModelParams03[0][3],Int)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_7128\580879155.py:5: OptimizeWarning: Unknown solver options: xtol
  res = minimize(SquaredError, g0, method='nelder-mead',options={'xtol': 1e-3, 'maxiter':100,'disp': False})


NameError: name 'ModelFit' is not defined